In [1]:
from pypdf import PdfReader
import re

In [2]:
complet_text: list[str] = []

for i in range (1,9):
    reader = PdfReader(f"debate/debate-{i}.pdf")

    page = reader.pages[0]

    text = page.extract_text()

    text = re.sub(r'\bC  ∫  Σ\b', "", text)
    
    text = re.sub(r'^\s*$\n', "", text, flags=re.MULTILINE)

    while re.search(r'(\w) (\w)', text):
        text = re.sub(r'(\w) (\w)', r'\1\2', text)

    complet_text.append(text)

In [3]:
for text in complet_text:
    print(text)

Chapa  Sigmóide 
Apresentação 
ciencia_da_computacao @ ufpa - belem 
Filosofia 
-  Simplicidade . ￿ 
-  Ubuntu . ￿ 
-  Colaboração . ￿ 
-  Rede  distribuída . ￿ 
-  Transparência . ￿ 
-  Presença  social . ￿ 
-  Amigável . ￿ 
1º  Tesoureiro 
Luiz  Henrique  Mendes  Costa 
Semestre :  2º 
Gosta  de : ￿ 
    -  animes ￿ 
-  manhwa ￿ 
-  games ￿ 
-  RPG ￿ 
-  cibersegurança ￿ 
-  curiosidades ￿ 
      tecnológicas ￿ henrique @ ubuntu 
 
1º  Secretário 
João  Davi  Costa  de  Souza 
Semestre :  2º 
Gosta  de : 
-  programação ￿ 
-  software  livre ￿ 
-  música ￿ 
-  sistemas ￿ 
      operacionais ￿ 
Environment : 
-  Hyprland  ( WM ) ￿ 
-  Kitty  ( terminal ) ￿ 
-  VS  Code  ( Editor ) 
jd @ arch 
 
Tesoureiro  Geral 
Yuri  Gabriel  Cardoso  Delgado 
OBS :  desenvolveu  o  estilo  de  apresentação 
Semestre :  2º 
Gosta  de : 
-  Linux ￿ 
-  Ficção  Científica ￿ 
-  Low - Level ￿ 
-  KPOP ￿ 
-  TRON ￿ 
Environment : 
-  Hyprland  ( WM ) ￿ 
-  ST  ( terminal ) ￿ 
-  Neovim  ( Editor ) 
vane

In [4]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300,
    chunk_overlap=10,
    separators=["\n\n", "\n", " ", ""],
)

chunks = text_splitter.create_documents(complet_text)

In [5]:
print(f'number of chunks: {len(chunks)}')

for i,chunk in enumerate(chunks):
    print(f'Chunk {i+1}: len={len(chunk.page_content)}')
    print(chunk.page_content)

number of chunks: 9
Chunk 1: len=67
Chapa  Sigmóide 
Apresentação 
ciencia_da_computacao @ ufpa - belem
Chunk 2: len=155
Filosofia 
-  Simplicidade . ￿ 
-  Ubuntu . ￿ 
-  Colaboração . ￿ 
-  Rede  distribuída . ￿ 
-  Transparência . ￿ 
-  Presença  social . ￿ 
-  Amigável . ￿
Chunk 3: len=208
1º  Tesoureiro 
Luiz  Henrique  Mendes  Costa 
Semestre :  2º 
Gosta  de : ￿ 
    -  animes ￿ 
-  manhwa ￿ 
-  games ￿ 
-  RPG ￿ 
-  cibersegurança ￿ 
-  curiosidades ￿ 
      tecnológicas ￿ henrique @ ubuntu
Chunk 4: len=263
1º  Secretário 
João  Davi  Costa  de  Souza 
Semestre :  2º 
Gosta  de : 
-  programação ￿ 
-  software  livre ￿ 
-  música ￿ 
-  sistemas ￿ 
      operacionais ￿ 
Environment : 
-  Hyprland  ( WM ) ￿ 
-  Kitty  ( terminal ) ￿ 
-  VS  Code  ( Editor ) 
jd @ arch
Chunk 5: len=289
Tesoureiro  Geral 
Yuri  Gabriel  Cardoso  Delgado 
OBS :  desenvolveu  o  estilo  de  apresentação 
Semestre :  2º 
Gosta  de : 
-  Linux ￿ 
-  Ficção  Científica ￿ 
-  Low - Level ￿ 
-  KPOP ￿ 
-  

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

model_name = "sentence-transformers/all-MiniLM-L6-v2"

embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device":"cpu"}
)

vector_store = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings,
    persist_directory="./chroma_db"
)

retriever = vector_store.as_retriever()

In [7]:
query = "What kind of environment does Yuri use?"

relevant_docs = retriever.invoke(query)

print(f"\n '{query}'")

for i, doc in enumerate(relevant_docs):
    print(f"Document {i+1} (Context):")
    print(doc.page_content)


 'What kind of environment does Yuri use?'
Document 1 (Context):
Tesoureiro  Geral 
Yuri  Gabriel  Cardoso  Delgado 
OBS :  desenvolveu  o  estilo  de  apresentação 
Semestre :  2º 
Gosta  de : 
-  Linux ￿ 
-  Ficção  Científica ￿ 
-  Low - Level ￿ 
-  KPOP ￿ 
-  TRON ￿ 
Environment : 
-  Hyprland  ( WM ) ￿ 
-  ST  ( terminal ) ￿ 
-  Neovim  ( Editor )
Document 2 (Context):
Vice - Presidente 
Lucas  Santos  Diniz 
Semestre :  2º 
Gosta  de : ￿ 
    -  Tecnologia ￿ 
-  Estudar ￿ 
-  Séries ￿ 
-  Filmes ￿ 
-  Jogos ￿ 
-  Desenhos ￿ 
Environment : 
-  Hyprland  ( WM ) ￿ 
-  Kitty  ( terminal ) ￿ 
-  VS  Code  ( Editor ) 
zost @ arch
Document 3 (Context):
1º  Secretário 
João  Davi  Costa  de  Souza 
Semestre :  2º 
Gosta  de : 
-  programação ￿ 
-  software  livre ￿ 
-  música ￿ 
-  sistemas ￿ 
      operacionais ￿ 
Environment : 
-  Hyprland  ( WM ) ￿ 
-  Kitty  ( terminal ) ￿ 
-  VS  Code  ( Editor ) 
jd @ arch
Document 4 (Context):
Secretário  Geral 
Semestre :  2º 
Gosta  de : ￿ 
   

In [8]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_ollama import OllamaLLM
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv

load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")

#llm = OllamaLLM(model="llama3")

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant. Use the following context to answer the question.
If you don't know the answer, say that you don't have enough information in the context.

Contexto:
{context}

Pergunta: {input}
""")

document_chain = create_stuff_documents_chain(
    llm=llm,
    prompt=prompt,
)

rag_chain = create_retrieval_chain(retriever, document_chain)

query = "Tell me What kind of environment does Mateus use?."

response = rag_chain.invoke({"input": query})

print(f"\nresponse: {response['answer']}")


response: Mateus uses the following environment:
*   **WM:** scroll
*   **Terminal:** foot
*   **Editor:** mousepad
